In [35]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

train_data=np.load("data_train.pkl",allow_pickle=True)
test_data=np.load("data_test.pkl",allow_pickle=True)

train_data=train_data
test_data=test_data
train_df=pd.DataFrame({'comments':train_data[0],'labels':train_data[1]})
test_df=pd.DataFrame({'comments':test_data})
#print(train_df)

#lowercasing all comments
train_df['comments']=train_df['comments'].str.lower() 
test_df['comments']=test_df['comments'].str.lower() 

train_df['sentence_token']=train_df.comments.apply(sent_tokenize)
train_df['sentence_count']=train_df.sentence_token.apply(len)
test_df['sentence_token']=test_df.comments.apply(sent_tokenize)
test_df['sentence_count']=test_df.sentence_token.apply(len)


#average word length
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

train_df['ave_word_len']=train_df.comments.apply(avg_word)
test_df['ave_word_len']=test_df.comments.apply(avg_word)

#remove special character
def remove_special_char(x):
    return re.sub("[^0-9a-zA-Z]+",' ',x)

train_df['comments']=train_df['comments'].apply(remove_special_char)
test_df['comments']=test_df['comments'].apply(remove_special_char)


In [36]:
import nltk
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

def remove_stopwords(x):
    result = ""
    for word in x.split():
        if word not in stopWords:
            result += word+" "
    return result

def count_stopwords(x):
    result = 0
    for word in x.split():
        if word in stopWords:
            result += 1
    return result


train_df['stopwords_count']=train_df['comments'].apply(count_stopwords)
train_df['comments']=train_df['comments'].apply(remove_stopwords)

test_df['stopwords_count']=test_df['comments'].apply(count_stopwords)
test_df['comments']=test_df['comments'].apply(remove_stopwords)

In [37]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

def lemmatize(x):
    result = ""
    for word in x.split(" "):
        result+=lemmatizer.lemmatize(word)+" "
    return result

train_df['comments']=train_df['comments'].apply(lemmatize)
test_df['comments']=test_df['comments'].apply(lemmatize)

In [38]:
train_df['token']=train_df.comments.apply(word_tokenize)
train_df['word_count']=train_df.token.apply(len)
train_df['ave_sentence_len']=train_df['word_count']/train_df['sentence_count']

test_df['token']=test_df.comments.apply(word_tokenize)
test_df['word_count']=test_df.token.apply(len)
test_df['ave_sentence_len']=test_df['word_count']/test_df['sentence_count']

In [39]:
test_df.head()


,comments,sentence_token,sentence_count,ave_word_len,stopwords_count,token,word_count,ave_sentence_len
0,trout bryant led league strikeout trout actual...,[trout and bryant have both led the league in ...,4,4.689655,15,"[trout, bryant, led, league, strikeout, trout,...",16,4.0
1,gt like estonian good reason fear russian expa...,[&gt; just like estonians have good reasons to...,2,5.583333,12,"[gt, like, estonian, good, reason, fear, russi...",25,12.5
2,sol primeval sotp oblivious find next episode,"[will sol_primeval sotp being oblivious?, find...",2,5.272727,5,"[sol, primeval, sotp, oblivious, find, next, e...",7,3.5
3,moving ostwald border back pre 1967 border dec...,[moving ostwald borders back to the pre 1967 b...,5,4.857143,71,"[moving, ostwald, border, back, pre, 1967, bor...",86,17.2
4,take bag morty,"[you have to take it out of the bag, morty!]",1,3.300000,7,"[take, bag, morty]",3,3.0


In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df, train_df.labels, test_size=0.33, random_state=0)

# X_train=train_df
# y_train=train_df.labels
# X_test=test_df

In [63]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
 

'''
transformer = Pipeline([
    ('features', FeatureUnion(transformer_list=[
        # Part 1
        ('comments', Pipeline([
            ('selector', TextSelector('comments'))#,('tfidf', TfidfVectorizer())
        ])),  # booleans close
        #('numericals', Pipeline([
        #    ('selector', TypeSelector(np.number)),
        #    ('scaler', StandardScaler()),
      #  ])),  # numericals close
    ])),  # features close
])  # pipeline close
'''

#transformer.fit(X_train,y_train)
#pred = transformer.predict(X_test)

"\ntransformer = Pipeline([\n    ('features', FeatureUnion(transformer_list=[\n        # Part 1\n        ('comments', Pipeline([\n            ('selector', TextSelector('comments'))#,('tfidf', TfidfVectorizer())\n        ])),  # booleans close\n        #('numericals', Pipeline([\n        #    ('selector', TypeSelector(np.number)),\n        #    ('scaler', StandardScaler()),\n      #  ])),  # numericals close\n    ])),  # features close\n])  # pipeline close\n"

In [85]:
transformer = Pipeline([
                        ('selector',TextSelector(key='comments')),
                        ('tfidf', TfidfVectorizer(use_idf=True, smooth_idf=1,sublinear_tf=True,norm='l2',))
])

In [84]:
print(transformer.fit_transform(X_train))

  (0, 21284)	0.1534557848387187
  (0, 40906)	0.23391233095307967
  (0, 34103)	0.11840624672860121
  (0, 20846)	0.12209788109706853
  (0, 37360)	0.16053947952497527
  (0, 46076)	0.16357724023745912
  (0, 21440)	0.25400914231967614
  (0, 45271)	0.14739973959839275
  (0, 16701)	0.1898468946534359
  (0, 28982)	0.2221365682407341
  (0, 52340)	0.28257764156606996
  (0, 52244)	0.2761430559106751
  (0, 23395)	0.3297569078541303
  (0, 30496)	0.368255689520253
  (0, 41179)	0.3946349875801853
  (0, 6528)	0.33533623768593446


In [86]:
from sklearn.preprocessing import MinMaxScaler

class NumberSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

transformer_sentence = Pipeline([
                        ('selector',NumberSelector(key='sentence_count')),
                        ('standard',MinMaxScaler())
])    

transformer_sentence.fit_transform(X_train)


transformer_word_count = Pipeline([
                        ('selector',NumberSelector(key='word_count')),
                        ('standard',MinMaxScaler())
])    

transformer_stopwords_count = Pipeline([
                        ('selector',NumberSelector(key='stopwords_count')),
                        ('standard',MinMaxScaler())
])  

transformer_stopwords_count = Pipeline([
                        ('selector',NumberSelector(key='stopwords_count')),
                        ('standard',MinMaxScaler())
]) 

transformer_ave_word_len = Pipeline([
                        ('selector',NumberSelector(key='ave_word_len')),
                        ('standard',MinMaxScaler())
]) 

transformer_ave_sentence_len = Pipeline([
                        ('selector',NumberSelector(key='ave_sentence_len')),
                        ('standard',MinMaxScaler())
]) 

transformer_sentence.fit_transform(X_train)
transformer_word_count.fit_transform(X_train)
transformer_stopwords_count.fit_transform(X_train)
transformer_ave_sentence_len.fit_transform(X_train)
transformer_ave_word_len.fit_transform(X_train)
transformer_sentence.fit_transform(X_test)
transformer_word_count.fit_transform(X_test)
transformer_stopwords_count.fit_transform(X_test)
transformer_ave_sentence_len.fit_transform(X_test)
transformer_ave_word_len.fit_transform(X_test)

array([[0.14360465],
       [0.09593481],
       [0.10195448],
       ...,
       [0.0869186 ],
       [0.09853575],
       [0.11138311]])

In [87]:
features=FeatureUnion([('comments',transformer)
                              ,('sentence_count',transformer_sentence)
                                ,('word_count',transformer_word_count)
                                  ,('stopwords_count',transformer_word_count)
                                  ,('ave_word_len',transformer_word_count)
                                  ,('ave_sentence_len',transformer_word_count)])

combine_features=Pipeline([('features',features)])
combine_features.fit_transform(X_train)
combine_features.fit_transform(X_test)

<23100x37711 sparse matrix of type '<class 'numpy.float64'>'
	with 553594 stored elements in Compressed Sparse Row format>

In [ ]:
# from sklearn.linear_model import SGDClassifier
# 'clf', SGDClassifier(loss='hinge',penalty='l2',alpha=1e-3,n_iter=5,random_state=42)

In [97]:
from sklearn.linear_model import SGDClassifier
#l2’;其余有’none’,‘l1’,‘elasticnet’
alpha_group = [0.00001,0.00005,0.0001,0.0002,0.0005]
penal_group = ['l2','none','l1','elasticnet']
#loss_group = ['hinge','log']
for penal in penal_group:
    clf=Pipeline([('features',features), 
                  ('classifier', SGDClassifier(loss='hinge',penalty=penal,alpha=0.00005,random_state=42))])

    clf.fit(X_train, y_train)

    pred = clf.predict(X_test)
    score = np.mean(pred==y_test)
    print(penal,"    accuracy:   %0.3f" % score)

l2     accuracy:   0.550
none     accuracy:   0.517
l1     accuracy:   0.434
elasticnet     accuracy:   0.538


In [91]:
from sklearn.svm import SVC
clf=Pipeline([('features',features), ('classifier', SVC(kernel='linear', C=1, gamma = 1))])

clf.fit(X_train, y_train)
 
pred = clf.predict(X_test)
score = np.mean(pred==y_test)
print("accuracy:   %0.3f" % score)

accuracy:   0.503


In [88]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import cross_val_score
# alpha_group=[0.05,0.1, 0.165, 0.2,0.25,0.3,0.4,0.5]
alpha_group=[0.165]
for alpha in alpha_group:
    classifier=Pipeline([('features',features),
                        ('classifier', MultinomialNB(alpha=alpha,fit_prior=True,class_prior=None))])

    classifier.fit(X_train,y_train)
    pred=classifier.predict(X_test)
    print(alpha, "\t" , np.mean(pred==y_test))

0.165 	 0.5595238095238095


baseline: Naive Bayes 0.5612142857142857
                      0.5313571428571429 (minlen = 5)
                      0.5433571428571429 (minilen =2, alpha =0.2)
                      0.5504285714285714 (minilen =2, alpha =0.4, 1-gram + 2-gram)
                      
MultinomialNB minilen =1, 1-gram + 2-gram, alpha=
0.1 	 0.549
0.165 	 0.5522857142857143
0.2 	 0.5515714285714286
0.25 	 0.5502857142857143
0.3 	 0.5507142857142857
0.4 	 0.5506428571428571
0.5 	 0.5500714285714285

baseline: linear SVM 0.503

In [73]:
print(pred)
np.savetxt('predictions_phase_2.csv',np.array(pred),delimiter='\n',newline='\n',fmt='%s')

['baseball' 'europe' 'anime' ... 'GlobalOffensive' 'gameofthrones' 'wow']


In [73]:
pd.set_option('display.max_colwidth', -1)

In [13]:
#X_train.head()

In [52]:
#from vectorizer import TfidfVectorizer

# vectorizer=TfidfVectorizer()
# X=vectorizer.fit_transform(X_train)
# y=y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier

start = time.time()

#for n_feature in [50, 80, 100, 150, 200]:
clf = RandomForestClassifier(n_estimators=20,max_features=100,min_impurity_decrease=0.0001,random_state=1)

clf.fit(X, y)

pred = clf.predict(vectorizer.transform(X_test))
score = metrics.accuracy_score(y_test, pred)
end = time.time()
elapsed_time = end - start
print("accuracy:   %0.3f" % score)
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

In [ ]:
from sklearn.linear_model import Perceptron


start = time.time()

clf = Perceptron(max_iter=50)
 
clf.fit(X, y)
 
pred = clf.predict(vectorizer.transform(X_test))
score = metrics.accuracy_score(y_test, pred)
end = time.time()
elapsed_time = end - start
print("accuracy:   %0.3f" % score)
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
 